This code is for looking for a single event's spacepoints and plotting them

In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly

In [2]:
def get_index(file_name, group_label, event_index):

    with h5py.File(file_name, 'r') as f:

        sp = f[group_label]

        sequence_info_sp = sp['sequence_info_filtered']

        output = np.where(sequence_info_sp[:,0] == event_index)[0][0]

        print(output)

        return output


In [12]:
def plot(file_name, og_file_name, group_label, index):

    with h5py.File(file_name, 'r') as f:

        sp = f[group_label]

        event_id, starting_index, num_points, run, subrun, event, description = sp['sequence_info_filtered'][index]
        points = sp['point_info_filtered'][starting_index:starting_index+num_points]

    with h5py.File(og_file_name, 'r') as f:

        pt = f['particle_table']
        et = f['event_table']
        sp = f['spacepoint_table']

        pt_sequence_info = pt['sequence_info']
        sp_sequence_info = sp['sequence_info']

        parent_id = pt['parent_id']
        g4_pdg = pt['g4_pdg']
        end_position = pt['end_position_corr']
        start_position = pt['start_position_corr']
        nu_vtx = et['nu_vtx']
        spacepoints = sp['position']

        event_index_pt, starting_index_pt, num_points_pt = pt_sequence_info[np.where(pt_sequence_info[:, 0] == event_id)][0]
        event_index_sp, starting_index_sp, num_points_sp = sp_sequence_info[np.where(sp_sequence_info[:, 0] == event_id)][0]

        print(event_index_pt, starting_index_pt, num_points_pt)
        print(event_index_sp, starting_index_sp, num_points_sp)

        og_positions = spacepoints[starting_index_sp:starting_index_sp+num_points_sp]

        x_og = og_positions[:,0]
        y_og = og_positions[:,1]
        z_og = og_positions[:,2]

        primary_indices = starting_index_pt + np.where(parent_id[starting_index_pt: starting_index_pt+num_points_pt] == 0)[0]

        primary_particles = g4_pdg[primary_indices].transpose()[0]
        primary_ends = end_position[primary_indices]
        # starts = start_position[starting_index_pt: starting_index_pt+num_points_pt]
        starts = start_position[primary_indices]
        nu_vertex = nu_vtx[np.where(pt_sequence_info[:, 0] == event_id)][0]



    print(primary_indices)
    print(primary_particles)
    print(primary_ends)
    print(starts)
    print(nu_vertex)

    x_p = primary_ends[:, 0]
    y_p = primary_ends[:, 1]
    z_p = primary_ends[:, 2]

    x_start = starts[:, 0]
    y_start = starts[:, 1]
    z_start = starts[:, 2]

    print(f'{starting_index}, {num_points}')
    print(f'{run}, {subrun}, {event}')
    print(f'{points[0]}')
    print(description)

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    integral = points[:,3]

    # fig = plt.figure(dpi=200)
    # ax = fig.add_subplot(projection='3d')
    # ax.scatter(x,y,z,c=integral, cmap='viridis', alpha=1, marker='.')
    # ax.set_xlabel('x [cm]')
    # ax.set_ylabel('y [cm]')
    # ax.set_zlabel('z [cm]')
    # ax.set_aspect('equal')
    # ax.view_init(elev=15, azim=160)
    # fig.tight_layout()

    df = pd.DataFrame({'x':x, 'y':y, 'z':z, 'q':integral})
    df_primary_ends = pd.DataFrame({'x':x_p, 'y':y_p, 'z':z_p, 'label':primary_particles})
   

    plotly_fig = px.scatter_3d(df, x='x', y='y', z='z', size='q', color='q',color_continuous_scale  = plotly.colors.sequential.Plasma)
    # plotly_fig.add_trace(go.Scatter3d(x=x_og, y=y_og, z=z_og, line=None, mode='markers', marker={'size':1}))
    # plotly_fig.add_trace(go.Scatter3d(x=x_p, y=y_p, z=z_p, hoverinfo='text', text=primary_particles, line=None, mode='markers', marker={'size':5}))
    # plotly_fig.add_trace(go.Scatter3d(x=x_start, y=y_start, z=z_start, line=None, mode='markers', marker={'size':3}))
    # plotly_fig.add_trace(go.Scatter3d(x=[nu_vertex[0]], y=[nu_vertex[1]], z=[nu_vertex[2]], hoverinfo='text', text='vertex', line=None, mode='markers', marker={'size':5}))
    plotly_fig.update_layout(
        width=2000,height=1800,
        # width=1000,height=900,
        # width=500,height=450,
        margin=dict(l=0, r=0, b=0, t=0),
        scene_camera=dict(
            eye=dict(x=-1.5, y=-1.1, z=0.5)),  # Position of the camera
        scene = dict(xaxis = dict(showgrid = False,showticklabels = False, visible=False),
                                        yaxis = dict(showgrid = False,showticklabels = False, visible=False),
                                        zaxis = dict(showgrid = False,showticklabels = False, visible=False)),
        # paper_bgcolor='rgba(0,0,0,0)',
        # plot_bgcolor='rgba(0,0,0,0)'
    )
    plotly_fig.update_traces(marker=dict(line=dict(width=0), colorscale='Viridis'))
    plotly_fig.show()
    # plotly_fig.write_image("transparent_plot.png", width=1500,height=1200, scale=2)


def plot_hist_volume(file_name, group, index):

    with h5py.File(file_name, 'r') as f:

        sp = f['spacepoint_table']

        starting_index, num_points = sp['metadata'][index]
        run, subrun, event = sp['event_id'][starting_index]
        points = sp['position'][starting_index:starting_index+num_points]

    print(f'{starting_index}, {num_points}')
    print(f'{run}, {subrun}, {event}')
    print(f'{points[0]}')

    print(points.transpose())

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    buffer = 10
    num_bins = 20
    x_bounds = np.linspace(min(x)-buffer, max(x)+buffer, num_bins)
    y_bounds = np.linspace(min(y)-buffer, max(y)+buffer, num_bins)
    z_bounds = np.linspace(min(z)-buffer, max(z)+buffer, num_bins)

    hist3d = np.histogramdd(points, bins=(x_bounds, y_bounds, z_bounds))[0]

    print(np.shape(hist3d))

    print(np.max(hist3d))

    X, Y, Z = np.meshgrid(np.diff(x_bounds), np.diff(y_bounds), np.diff(z_bounds))

    print(np.shape(X))

    fig = go.Figure(data=go.Volume(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=hist3d.flatten(),
        isomin=0.1,
        isomax=0.8,
        opacity=0.1, # needs to be small to see through all surfaces
        surface_count=3, # needs to be a large number for good volume rendering
        ))
    fig.show()


In [133]:
print(get_index('../eta-pi-data/eta-pi0-update.h5', '1 Eta', 15913))

5590


In [21]:
index = get_index('../eta-pi-data/eta-pi0-update.h5', '1 Eta', 25959)
plot('../eta-pi-data/eta-pi0-update.h5', '../eta-pi-data/merged_Eta_update_sequenced.h5', '1 Eta', index)

9060
25959 5770542 717
25959 11838573 1214
[5770542 5770543 5770544 5770545 5770546 5770547 5770548]
[  13   22   22  211 2112 -211  111]
[[  74.98363  -156.76616   887.3096  ]
 [  11.242399  -61.148827  503.62656 ]
 [  24.43062   -46.0993    472.1955  ]
 [  -9.447445  -24.699547  559.28424 ]
 [  10.176832  -51.073353  509.42935 ]
 [  14.461147  -47.76316   474.80142 ]
 [   9.089955  -48.62601   472.70496 ]]
[[  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]
 [  9.089962 -48.62601  472.70496 ]]
[ 87.432076  95.052376 940.60614 ]
3376711, 479
17133, 10, 548
[ 65.08489  -13.406022 448.8       44.839085]
1


In [112]:
plot('../eta-pi-data/eta-pi0-update.h5', '../eta-pi-data/merged_NCPi0_update.h5', '0 NCPi0', 11)

12 4834 442
12 10684 986
[4834 4835 4836]
[2112 2212  111]
[[158.62305   37.39063  270.65564 ]
 [152.12982   29.498543 246.40514 ]
 [154.46765   30.457283 245.04256 ]]
[[154.46765   30.457283 245.04256 ]
 [154.46765   30.457283 245.04256 ]
 [154.46765   30.457283 245.04256 ]]
[150.99283   34.585426 245.16612 ]
3074, 274
7022, 223, 11163
[153.30734   26.604351 249.9       70.609436]
10
